In [1]:
!pip install --upgrade transformers
!pip install qwen_vl_utils
!pip install flash-attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 64.6 MB/s eta 0:00:0000:010:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.1
    Uninstalling transformers-4.45.1:
      Successfully uninstalled transformers-4.45.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 52.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 21.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.5 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl size=187315346 sha256=6ebfbdcbdd164f80278a954d29a1bc9d620130264215f0fb93374a6ab4e0a283
  Stored in directory: /root/.cache/pip/wheels/7e/e3/c3/89c7a2f3c4adc07cd1c675f8bb7b9ad4d18f64a72bccdfe826
Successfully built flash-attn


In [14]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch
from flash_attn.modules.mha import FlashSelfAttention

device = "cuda" if torch.cuda.is_available() else "cpu"

model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", device_map="auto"
)
model.config.attention_mechanism = "flash_attention"

processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
%%time
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "/kaggle/input/5643231321/photo_2024-11-09_14-59-28.jpg",
            },
            {
                "type": "text",
                "text": "Посмотри на изображение, на котором изображен график улиц города. Зелёные линии представляют участки с низким уровнем пешеходной загруженности, а красные линии — самые переполненные участки для пешеходов. Посмотри на ситуацию и предложи несколько альтернативных вариантов для перераспределения пешеходов, чтобы уменьшить нагрузку на самые загруженные участки. Подумай, какие меры можно предпринять, чтобы улучшить пешеходную инфраструктуру и распределить пешеходный поток более равномерно, например, улучшение пешеходных переходов, создание новых пешеходных маршрутов или увеличение доступности общественных зон.  Опиши все текстом."
            },
        ],
    }
]

text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to(device)

generated_ids = model.generate(**inputs, max_new_tokens=512)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

['Извините, но как текстовый AI, я не могу предоставить изображения или карты. Однако, я могу предложить несколько альтернативных вариантов для перераспределения пешеходов, чтобы уменьшить нагрузку на самые загруженные участки.\n\n1. **Улучшение пешеходных переходов**: Создание новых пешеходных переходов или реконструкция существующих может помочь уменьшить нагрузку на самые переполненные участки. Это может включать в себя установку дополнительных светофоров, увеличение количества пешеходных переходов или создание специальных пешеходных переходов для уменьшения времени ожидания.\n\n2. **Создание новых пешеходных маршрутов**: Создание новых пешеходных маршрутов может помочь уменьшить нагрузку на самые переполненные участки. Это может включать в себя создание новых пешеходных путей или реконструкцию существующих путей.\n\n3. **Увеличение доступности общественных зон**: Создание новых общественных зон или увеличение доступности существующих может помочь уменьшить нагрузку на самые перепол

In [16]:
lines = output_text[0].split('\n')
lines = lines[2:]
formatted_lines = []
for line in lines:
    parts = line.split(': ')
    if len(parts) > 1:
        formatted_lines.append(f"{parts[0]}: {parts[1].split('. ')[0]}.")
    else:
        formatted_lines.append(line)

formatted_text = '\n'.join(formatted_lines)

print(formatted_text)

1. **Улучшение пешеходных переходов**: Создание новых пешеходных переходов или реконструкция существующих может помочь уменьшить нагрузку на самые переполненные участки.

2. **Создание новых пешеходных маршрутов**: Создание новых пешеходных маршрутов может помочь уменьшить нагрузку на самые переполненные участки.

3. **Увеличение доступности общественных зон**: Создание новых общественных зон или увеличение доступности существующих может помочь уменьшить нагрузку на самые переполненные участки.

4. **Улучшение инфраструктуры**: Улучшение инфраструктуры, включая дороги, общественные зоны и транспортные системы, может помочь уменьшить нагрузку на самые переполненные участки.

5. **Улучшение пешеходной инфраструктуры**: Улучшение пешеходной инфраструктуры, включая создание новых пешеходных переходов, увеличение количества светофоров или создание новых пешеходных путей, может помочь уменьшить нагрузку на самые переполненные.
